# Clustering Neighborhoods in Toronto 
### Part 1 : Wiki Scrap

Install Beautifulsoup Module

In [1]:
!pip install beautifulsoup4

Import necessary libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np  

I will use the BeautifulSoup module with the Python’s html.parser to scrape the wiki page and read into a Pandas Dataframe.   
Then I will remove "Not assigned" Boroughs (dropping the rows) and reset the index.  
Finally check any Neighbourhoods "Not assigned" and replace the value with the Borough value

In [3]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
content = r.text
soup = BeautifulSoup(content, "html.parser")
table = soup.find_all('table',class_ = 'wikitable')[0]
df = pd.read_html(str(table))[0]
df = df[df.Borough != "Not assigned"].reset_index(drop=True) 
df.Neighbourhood = df.Neighbourhood.where(df.Neighbourhood != "Not assigned", df.Borough.values)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

### Incorporate latitude/longitud 

Geocoder seems not to be yet compatible with the new 3.7 version of Python so I will go for the CVS file (I avoid posting the error as it was too long and made the rest of the coding hard to follow).  

In [5]:
!pip install wget
import wget
wget.download('http://cocl.us/Geospatial_data', 'coords.csv')

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=6c5135547bffe604d8b3930adb1100811bb647807c34711a2ddb3dd8e4ed8e73
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


'coords.csv'

Read the coords file into a pandas dataframe

In [6]:
coords = pd.read_csv('coords.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge both files

In [7]:
dfcoords = pd.merge(df, coords, left_on=['Postal Code'],right_on=['Postal Code'],how='left')
dfcoords.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 2 - Exploring DataSet

In [8]:
!pip install folium

from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

     |████████████████████████████████| 93 kB 3.3 MB/s  eta 0:00:01


I will analyze boroughs containing the word Toronto and then replicate

In [9]:
toronto = dfcoords[dfcoords['Borough'].str.contains('Toronto')]
toronto.reset_index(drop=True, inplace=True)
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Let´s create a map with the average latitude and longitude of all Neighbourhoods

In [10]:
latitude = toronto.Latitude.mean()
longitude = toronto.Longitude.mean()
map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        
      parse_html=False).add_to(map)
map

#### Like in the New York example I will explore venues. I will choose the Harbourfront neighbourhood as I assume there will be good places to visit there

In [11]:
# The code was removed by Watson Studio for sharing.

#### Search for venue and define the corresponding URL

In [12]:
lat = toronto[toronto['Neighbourhood'].str.contains('Harbourfront')].Latitude.values[0]
long = toronto[toronto['Neighbourhood'].str.contains('Harbourfront')].Longitude.values[0]

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{long}&radius=500&limit=100&v={VERSION}'
results = requests.get(url).json()

#### Reusing existing function to get category venues

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
from pandas import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("There are",len(nearby_venues), "venues in Harbourfront. Below the first 5")
nearby_venues.head()

There are 44 venues in Harbourfront. Below the first 5


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


#### Let´s create a map with the venues and pop ups with its categories

In [15]:
# create map for Harbourfront
latitude = nearby_venues.lat.mean()
longitude = nearby_venues.lng.mean()
harbour_map = folium.Map(location=[latitude, longitude], zoom_start=16)


# instantiate a feature group for the venues in the dataframe
venues = folium.map.FeatureGroup()

# loop through the 44 venues and add each to the feature group
for lat, lng, in zip(nearby_venues['lat'], nearby_venues['lng']):
    venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text with category venue to each marker on the map
latitudes = list(nearby_venues.lat)
longitudes = list(nearby_venues.lng)
labels = list(nearby_venues.categories)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(harbour_map)    
    
# add venues to map and display it
harbour_map.add_child(venues)


### Let´s check the top 5 Category Venues by their count

In [19]:
nearby_venues[['categories','name']].groupby('categories') \
             .count() \
             .sort_values(['name'], ascending=False) \
             .reset_index() \
             .rename(columns={'name': 'count'}) \
             .head(5)

,categories,count
0,Coffee Shop,8
1,Bakery,3
2,Pub,3
3,Park,3
4,Theater,2


### Part 3a - Analyzing neighborhoods

#### Let's bring the function from the lab to repeat the same process to all the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{long}&radius=500&limit=100&v={VERSION}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
# utilizing the above function
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [28]:
# Let´s check the dataset and shape
print(toronto_venues.shape)
toronto_venues.head()

(928, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Let´s see how many venues each neighbourhood has

In [37]:
toronto_venues[['Neighbourhood', 'Venue Category']].groupby('Neighbourhood') \
             .count() \
             .sort_values(['Venue Category'], ascending=False) \
             .rename(columns={'Venue Category': 'count'})           

,count
Neighbourhood,
"Kensington Market, Chinatown, Grange Park",49
"Little Portugal, Trinity",48
"Regent Park, Harbourfront",44
"Toronto Dominion Centre, Design Exchange",44
"Richmond, Adelaide, King",43
"Runnymede, Swansea",41
St. James Town,41
"Garden District, Ryerson",40
Stn A PO Boxes,37


#### Let's find out how many unique categories can be curated from all the returned venues


In [38]:
unique = len(toronto_venues['Venue Category'].unique())
print(f'There are {unique} uniques categories.')

There are 115 uniques categories.


#### We can now analyze the neighbourhoods

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Bagel Shop,...,Supermarket,Sushi Restaurant,Tapas Restaurant,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# checking size
toronto_onehot.shape

(928, 116)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Bagel Shop,...,Supermarket,Sushi Restaurant,Tapas Restaurant,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,...,0.0,0.071429,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.032258,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.032258,0.0,0.0,0.0,0.0,0.032258


In [43]:
# confirm size again
toronto_grouped.shape

(39, 116)

#### Top 5 most common venue of each neighborhood

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
           venue  freq
0    Coffee Shop  0.17
1            Bar  0.17
2           Park  0.17
3  Movie Theater  0.17
4          Plaza  0.17


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0        Boat or Ferry   1.0
1  American Restaurant   0.0
2                 Pier   0.0
3             Pharmacy   0.0
4            Pet Store   0.0


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0                  Park  0.14
1       Auto Dealership  0.07
2  Fast Food Restaurant  0.07
3            Restaurant  0.07
4      Sushi Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0             Café  0.33
1  Harbor / Marina  0.17
2             Pier  0.17
3       Restaurant  0.17
4        Disc Golf  0.17


----Central Bay Street----
            venue  freq
0     Coffee Shop 

#### Let´s put the output into a Pandas Dataframe

In [45]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Boat or Ferry,Movie Theater,Coffee Shop,Plaza,Park,Bar,Discount Store,Disc Golf,Dessert Shop,Yoga Studio
1,"Brockton, Parkdale Village, Exhibition Place",Boat or Ferry,Yoga Studio,Electronics Store,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Department Store
2,"Business reply mail Processing Centre, South C...",Park,Performing Arts Venue,Sushi Restaurant,Pool,Coffee Shop,Farm,Farmers Market,Auto Dealership,Fast Food Restaurant,Thai Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Pier,Harbor / Marina,Restaurant,Disc Golf,Distribution Center,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop
4,Central Bay Street,Coffee Shop,Park,Pharmacy,Restaurant,Breakfast Spot,Pet Store,Pub,Pool,Café,Food Truck


### Part 3b - Clustering Neighbourhoods

In [47]:
from sklearn.cluster import KMeans

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 2, 4, 0, 0, 1, 4, 0, 0, 0], dtype=int32)

#### Now create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [48]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Theater,Café,Breakfast Spot,Yoga Studio,Mexican Restaurant,Brewery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Park,Fast Food Restaurant,Restaurant,Pool,Auto Dealership,Sushi Restaurant,Performing Arts Venue,Thai Restaurant,Pharmacy
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Park,Breakfast Spot,Restaurant,Café,Pharmacy,Food Truck,Performing Arts Venue,Pub,Pool
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Theater,Bakery,Performing Arts Venue,Dessert Shop,Liquor Store,Breakfast Spot,Brewery,Pub,Historic Site
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Pet Store,New American Restaurant,Mexican Restaurant,Liquor Store,Grocery Store


#### Finally, let´s visualize the result and take a break

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Let´s examine the discrimanting factor of the first  clusters (Cluster 0) which has the most venues

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Theater,Café,Breakfast Spot,Yoga Studio,Mexican Restaurant,Brewery
2,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Restaurant,Café,Pharmacy,Food Truck,Performing Arts Venue,Pub,Pool
3,Downtown Toronto,0,Coffee Shop,Theater,Bakery,Performing Arts Venue,Dessert Shop,Liquor Store,Breakfast Spot,Brewery,Pub,Historic Site
4,East Toronto,0,Pub,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Pet Store,New American Restaurant,Mexican Restaurant,Liquor Store,Grocery Store
5,Downtown Toronto,0,Boat or Ferry,Movie Theater,Coffee Shop,Plaza,Park,Bar,Discount Store,Disc Golf,Dessert Shop,Yoga Studio
6,Downtown Toronto,0,Coffee Shop,Park,Pharmacy,Restaurant,Breakfast Spot,Pet Store,Pub,Pool,Café,Food Truck
8,Downtown Toronto,0,Coffee Shop,Theater,Bakery,Performing Arts Venue,Italian Restaurant,Brewery,Café,Pet Store,Park,Chocolate Shop
10,Downtown Toronto,0,Theater,Plaza,Park,Nightclub,Boat or Ferry,Yoga Studio,Discount Store,Concert Hall,Convenience Store,Cosmetics Shop
11,West Toronto,0,Park,Boat or Ferry,Theater,Café,Art Gallery,Coffee Shop,Bakery,Kids Store,Performing Arts Venue,Paper / Office Supplies Store
12,East Toronto,0,Coffee Shop,Pub,Spa,Yoga Studio,Falafel Restaurant,Pharmacy,Pet Store,New American Restaurant,Liquor Store,Grocery Store


#### Coffe Shops and Pubs seem to be the outstanding feature of this cluster